In [19]:
pip install pandas mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd

# Load dataset
file_path = "C:/Users/Sairam/Downloads/climate_data (1).csv"
df = pd.read_csv(file_path)

# Display first few rows
print("Dataset Preview:")
print(df.head())


Dataset Preview:
         Country  Year  Avg_Temperature_C  CO2_Emissions_MMT  \
0  United States  2000              17.61            2671.31   
1  United States  2001              17.28            3098.08   
2  United States  2002              17.30            1895.17   
3  United States  2005              29.63            1078.40   
4  United States  2008              24.81             327.61   

   Deforestation_Rate_Percent  
0                        2.22  
1                        0.78  
2                        2.33  
3                        2.62  
4                        3.08  


In [24]:
# Data Cleaning
df.dropna(inplace=True)  # Remove missing values
df["year"] = df["year"].astype(int)  # Convert year to integer
df["avg_temperature"] = df["avg_temperature"].astype(float)  # Convert temperature to float
df["co2_emissions"] = df["co2_emissions"].astype(float)  # Convert CO2 emissions to float
df["deforestation_rate"] = df["deforestation_rate"].astype(float)  # Convert deforestation rate to float

# Print summary
print("\nCleaned Data Summary:")
print(df.info())



Cleaned Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             303 non-null    object 
 1   year                303 non-null    int64  
 2   avg_temperature     303 non-null    float64
 3   co2_emissions       303 non-null    float64
 4   deforestation_rate  303 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 12.0+ KB
None


In [33]:
# Rename columns
df.rename(columns={
    "Avg_Temperature_C": "avg_temperature",
    "CO2_Emissions_MMT": "co2_emissions",
    "Country": "country",
    "Deforestation_Rate_Percent": "deforestation_rate",
    "Year": "year"
}, inplace=True)

In [25]:
import mysql.connector

# Database Connection Configuration
db_config = {
    "host": "localhost",      # Change if MySQL runs on another server
    "user": "root",  # Replace with MySQL username
    "password": "9949237758",  # Replace with MySQL password
    "database": "climate_db"  # Ensure this database exists
}

# Establish Connection
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

print("Connected to MySQL successfully.")


Connected to MySQL successfully.


In [26]:
# Insert Data into MySQL
insert_query = """
INSERT INTO climate_data (country, year, avg_temperature, co2_emissions, deforestation_rate)
VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE 
    avg_temperature = VALUES(avg_temperature),
    co2_emissions = VALUES(co2_emissions),
    deforestation_rate = VALUES(deforestation_rate);
"""

data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Insert data in batches
cursor.executemany(insert_query, data_tuples)
conn.commit()

print(f"Inserted {cursor.rowcount} rows into MySQL.")


Inserted 303 rows into MySQL.


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib


In [41]:

# Load Data (Assuming CSV file format)
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")  # Replace with your file path

# Rename columns
data.rename(columns={
    "Avg_Temperature_C": "avg_temperature",
    "CO2_Emissions_MMT": "co2_emissions",
    "Country": "country",
    "Deforestation_Rate_Percent": "deforestation_rate",
    "Year": "year"
}, inplace=True)

# Encode Categorical Variable (Country)
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["country"])

# Selecting Features and Target Variables
X = data[["Country_Encoded", "year"]]
y_temp = data["avg_temperature"]
y_co2 = data["co2_emissions"]
y_deforestation = data["deforestation_rate"]

# Split Data
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X, y_co2, test_size=0.2, random_state=42)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X, y_deforestation, test_size=0.2, random_state=42)

# Train Models
temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)
deforestation_model = RandomForestRegressor(n_estimators=100, random_state=42)

temp_model.fit(X_train, y_temp_train)
co2_model.fit(X_train, y_co2_train)
deforestation_model.fit(X_train, y_deforestation_train)

# Save Models
joblib.dump(temp_model, "temperature_model.pkl")
joblib.dump(co2_model, "co2_model.pkl")
joblib.dump(deforestation_model, "deforestation_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Models trained and saved successfully!")


Models trained and saved successfully!


In [44]:
# Prediction Function
def predict_climate(country, year):
    # Load models and encoder
    temp_model = joblib.load("temperature_model.pkl")
    co2_model = joblib.load("co2_model.pkl")
    deforestation_model = joblib.load("deforestation_model.pkl")
    label_encoder = joblib.load("label_encoder.pkl")

    # Convert Country to Encoded Value
    country_encoded = label_encoder.transform([country])[0]

    # Prepare Input
    input_data = np.array([[country_encoded, year]])

    # Make Predictions
    predicted_temp = temp_model.predict(input_data)[0]
    predicted_co2 = co2_model.predict(input_data)[0]
    predicted_deforestation = deforestation_model.predict(input_data)[0]

    # Display Results
    return {
        "Predicted Temperature (°C)": round(predicted_temp, 2),
        "Predicted CO2 Emissions (MMT)": round(predicted_co2, 2),
        "Predicted Deforestation Rate (%)": round(predicted_deforestation, 2),
    }

# Example Usage
country_name = input("🌍 Enter Country: ")
year_input = int(input("📅 Enter Year: "))

prediction = predict_climate(country_name, year_input)
print("📊 Predicted Climate Impact:")
print(prediction)

📊 Predicted Climate Impact:
{'Predicted Temperature (°C)': np.float64(24.35), 'Predicted CO2 Emissions (MMT)': np.float64(1328.76), 'Predicted Deforestation Rate (%)': np.float64(2.51)}


c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Load dataset
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")  # Update path if needed

# Encode categorical variable (Country)
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["Country"])

# Selecting features and target variables
X = data[["Country_Encoded", "Year"]]
y_temp = data["Avg_Temperature_C"]
y_co2 = data["CO2_Emissions_MMT"]
y_deforestation = data["Deforestation_Rate_Percent"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data (80% train, 20% test)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X_scaled, y_temp, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X_scaled, y_co2, test_size=0.2, random_state=42)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X_scaled, y_deforestation, test_size=0.2, random_state=42)

# Train models
temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)
deforestation_model = RandomForestRegressor(n_estimators=100, random_state=42)

temp_model.fit(X_train, y_temp_train)
co2_model.fit(X_train, y_co2_train)
deforestation_model.fit(X_train, y_deforestation_train)

# Save models and scaler
joblib.dump(temp_model, "temperature_model.pkl")
joblib.dump(co2_model, "co2_model.pkl")
joblib.dump(deforestation_model, "deforestation_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Models trained and saved successfully!")

# Function to evaluate model performance
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"--- Model Performance for {target_name} ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print("-" * 40)

# Evaluate models
evaluate_model(temp_model, X_test, y_temp_test, "Temperature")
evaluate_model(co2_model, X_test, y_co2_test, "CO2 Emissions")
evaluate_model(deforestation_model, X_test, y_deforestation_test, "Deforestation Rate")

# Check feature importance
print("Feature Importance (Country_Encoded, Year):")
print("Temperature Model:", temp_model.feature_importances_)
print("CO2 Model:", co2_model.feature_importances_)
print("Deforestation Model:", deforestation_model.feature_importances_)

# Test predictions for a sample country and different years
sample_country = data["Country"].iloc[0]
country_encoded = label_encoder.transform([sample_country])[0]
test_years = [2025, 2050, 2075, 2100]
print(f"\nSample Predictions for {sample_country}:")
for year in test_years:
    input_data = scaler.transform(np.array([[country_encoded, year]]))
    temp_pred = temp_model.predict(input_data)[0]
    co2_pred = co2_model.predict(input_data)[0]
    def_pred = deforestation_model.predict(input_data)[0]
    print(f"Year {year}: Temp={temp_pred:.2f}°C, CO2={co2_pred:.2f} ppm, Deforestation={def_pred:.2f}%")

✅ Models trained and saved successfully!
--- Model Performance for Temperature ---
Mean Absolute Error (MAE): 7.19
Mean Squared Error (MSE): 75.10
Root Mean Squared Error (RMSE): 8.67
R² Score: -0.38
----------------------------------------
--- Model Performance for CO2 Emissions ---
Mean Absolute Error (MAE): 1165.44
Mean Squared Error (MSE): 1991355.69
Root Mean Squared Error (RMSE): 1411.15
R² Score: -0.13
----------------------------------------
--- Model Performance for Deforestation Rate ---
Mean Absolute Error (MAE): 1.43
Mean Squared Error (MSE): 2.76
Root Mean Squared Error (RMSE): 1.66
R² Score: -0.21
----------------------------------------
Feature Importance (Country_Encoded, Year):
Temperature Model: [0.4592159 0.5407841]
CO2 Model: [0.49035023 0.50964977]
Deforestation Model: [0.45590308 0.54409692]

Sample Predictions for United States:
Year 2025: Temp=18.06°C, CO2=2203.68 ppm, Deforestation=2.23%
Year 2050: Temp=18.06°C, CO2=2203.68 ppm, Deforestation=2.23%
Year 2075: T

c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Load Data
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")  # Update path if using augmented data

# Encode Categorical Variable (Country)
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["Country"])

# Selecting Features and Target Variables
X = data[["Country_Encoded", "Year"]]
y_temp = data["Avg_Temperature_C"]
y_co2 = data["CO2_Emissions_MMT"]
y_deforestation = data["Deforestation_Rate_Percent"]

# Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split Data (80% Train, 20% Test)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X_scaled, y_temp, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X_scaled, y_co2, test_size=0.2, random_state=42)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X_scaled, y_deforestation, test_size=0.2, random_state=42)

# Train Models
temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)
deforestation_model = RandomForestRegressor(n_estimators=100, random_state=42)

temp_model.fit(X_train, y_temp_train)
co2_model.fit(X_train, y_co2_train)
deforestation_model.fit(X_train, y_deforestation_train)

# Save Models and Scaler
joblib.dump(temp_model, "temperature_model.pkl")
joblib.dump(co2_model, "co2_model.pkl")
joblib.dump(deforestation_model, "deforestation_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Models trained and saved successfully!")

# Evaluate Models
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"--- Model Performance for {target_name} ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print("-" * 40)

evaluate_model(temp_model, X_test, y_temp_test, "Temperature")
evaluate_model(co2_model, X_test, y_co2_test, "CO2 Emissions")
evaluate_model(deforestation_model, X_test, y_deforestation_test, "Deforestation Rate")

# Check Feature Importance
print("Feature Importance (Country_Encoded, Year):")
print("Temperature Model:", temp_model.feature_importances_)
print("CO2 Model:", co2_model.feature_importances_)
print("Deforestation Model:", deforestation_model.feature_importances_)

✅ Models trained and saved successfully!
--- Model Performance for Temperature ---
Mean Absolute Error (MAE): 7.19
Mean Squared Error (MSE): 75.10
Root Mean Squared Error (RMSE): 8.67
R² Score: -0.38
----------------------------------------
--- Model Performance for CO2 Emissions ---
Mean Absolute Error (MAE): 1165.44
Mean Squared Error (MSE): 1991355.69
Root Mean Squared Error (RMSE): 1411.15
R² Score: -0.13
----------------------------------------
--- Model Performance for Deforestation Rate ---
Mean Absolute Error (MAE): 1.43
Mean Squared Error (MSE): 2.76
Root Mean Squared Error (RMSE): 1.66
R² Score: -0.21
----------------------------------------
Feature Importance (Country_Encoded, Year):
Temperature Model: [0.4592159 0.5407841]
CO2 Model: [0.49035023 0.50964977]
Deforestation Model: [0.45590308 0.54409692]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Step 1: Inspect the Dataset
print("Inspecting the Dataset...")
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")

# Basic info
print("Dataset Info:")
print(data.info())
print("\nYear Range:", data["Year"].min(), "to", data["Year"].max())
print("\nUnique Countries:", data["Country"].nunique())
print("\nMissing Values:")
print(data.isnull().sum())

# Check trends by year
print("\nAverage Temperature by Year:")
print(data.groupby("Year")["Avg_Temperature_C"].mean().round(2))
print("\nAverage CO2 Emissions by Year:")
print(data.groupby("Year")["CO2_Emissions_MMT"].mean().round(2))
print("\nAverage Deforestation_Rate_Percent Rate by Year:")
print(data.groupby("Year")["Deforestation_Rate_Percent"].mean().round(2))

# Step 2: Augment the Dataset
print("\nAugmenting the Dataset...")
countries = data["Country"].unique()
augmented_data = []
temp_increase_per_year = 0.02  # °C per year
co2_increase_per_year = 5      # MMT per year
def_increase_per_year = 0.1    # % per year

for country in countries:
    country_data = data[data["Country"] == country]
    if country_data.empty:
        continue
    base_year = country_data["Year"].max()
    base_data = country_data[country_data["Year"] == base_year].iloc[0]

    # Past years (1990 to base_year-1)
    for year in range(1990, base_year):
        years_diff = base_year - year
        augmented_data.append({
            "Country": country,
            "Year": year,
            "Avg_Temperature_C": base_data["Avg_Temperature_C"] - temp_increase_per_year * years_diff,
            "CO2_Emissions_MMT": max(0, base_data["CO2_Emissions_MMT"] - co2_increase_per_year * years_diff),
            "Deforestation_Rate_Percent": max(0, base_data["Deforestation_Rate_Percent"] - def_increase_per_year * years_diff)
        })

    # Future years (base_year+1 to 2100)
    for year in range(base_year + 1, 2101):
        years_diff = year - base_year
        augmented_data.append({
            "Country": country,
            "Year": year,
            "Avg_Temperature_C": base_data["Avg_Temperature_C"] + temp_increase_per_year * years_diff,
            "CO2_Emissions_MMT": base_data["CO2_Emissions_MMT"] + co2_increase_per_year * years_diff,
            "Deforestation_Rate_Percent": base_data["Deforestation_Rate_Percent"] + def_increase_per_year * years_diff
        })

# Combine and save
augmented_df = pd.DataFrame(augmented_data)
data = pd.concat([data, augmented_df], ignore_index=True)
data = data.dropna()

# Add noise to avoid overfitting
np.random.seed(42)
data["Avg_Temperature_C"] += np.random.normal(0, 0.1, len(data))
data["CO2_Emissions_MMT"] += np.random.normal(0, 5, len(data))
data["Deforestation_Rate_Percent"] += np.random.normal(0, 0.05, len(data))

data.to_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\augmented_climate_data.csv", index=False)
print("Augmented dataset saved as 'augmented_climate_data.csv'")

# Reload augmented dataset
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\augmented_climate_data.csv")

# Verify augmentation
print("\nAugmented Dataset Info:")
print(data.info())
print("\nYear Range:", data["Year"].min(), "to", data["Year"].max())
print("\nAverage Temperature by Year (sample):")
print(data.groupby("Year")["Avg_Temperature_C"].mean().round(2).head(10))
print("\nAverage CO2 Emissions by Year (sample):")
print(data.groupby("Year")["CO2_Emissions_MMT"].mean().round(2).head(10))
print("\nAverage Deforestation_Rate_Percent Rate by Year (sample):")
print(pd['deforestation_rate'].dtype())
print(data.groupby("Year")["Deforestation_Rate_Percent"].mean().round(2).head(10))

# Step 3: Preprocess Data
# Encode categorical variable (Country)
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["Country"])

# Selecting features and target variables
X = data[["Country_Encoded", "Year"]]
y_temp = data["Avg_Temperature_C"]
y_co2 = data["CO2_Emissions_MMT"]
y_deforestation = data["Deforestation_Rate_Percent"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data (80% train, 20% test)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X_scaled, y_temp, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X_scaled, y_co2, test_size=0.2, random_state=42)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X_scaled, y_deforestation, test_size=0.2, random_state=42)

# Step 4: Train Models
temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)
deforestation_model = RandomForestRegressor(n_estimators=100, random_state=42)

temp_model.fit(X_train, y_temp_train)
co2_model.fit(X_train, y_co2_train)
deforestation_model.fit(X_train, y_deforestation_train)

# Step 5: Evaluate Models
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"--- Model Performance for {target_name} ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print("-" * 40)

print("\nEvaluating Models...")
evaluate_model(temp_model, X_test, y_temp_test, "Temperature")
evaluate_model(co2_model, X_test, y_co2_test, "CO2 Emissions")
evaluate_model(deforestation_model, X_test, y_deforestation_test, "Deforestation_Rate_Percent Rate")

# Check feature importance
print("Feature Importance (Country_Encoded, Year):")
print("Temperature Model:", temp_model.feature_importances_)
print("CO2 Model:", co2_model.feature_importances_)
print("Deforestation_Rate_Percent Model:", deforestation_model.feature_importances_)

# Test predictions for United States
sample_country = "United States"
try:
    country_encoded = label_encoder.transform([sample_country])[0]
    test_years = [2025, 2050, 2075, 2100]
    print(f"\nSample Predictions for {sample_country}:")
    for year in test_years:
        input_data = scaler.transform(np.array([[country_encoded, year]]))
        temp_pred = temp_model.predict(input_data)[0]
        co2_pred = co2_model.predict(input_data)[0]
        def_pred = deforestation_model.predict(input_data)[0]
        print(f"Year {year}: Temp={temp_pred:.2f}°C, CO2={co2_pred:.2f} MMT, Deforestation_Rate_Percent={def_pred:.2f}%")
except ValueError:
    print(f"Country '{sample_country}' not found in dataset")

# Step 6: Save Models and Scalers
joblib.dump(temp_model, "temperature_model.pkl")
joblib.dump(co2_model, "co2_model.pkl")
joblib.dump(deforestation_model, "deforestation_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Models trained and saved successfully!")

Inspecting the Dataset...
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country                     303 non-null    object 
 1   Year                        303 non-null    int64  
 2   Avg_Temperature_C           303 non-null    float64
 3   CO2_Emissions_MMT           303 non-null    float64
 4   Deforestation_Rate_Percent  303 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 12.0+ KB
None

Year Range: 2000 to 2024

Unique Countries: 20

Missing Values:
Country                       0
Year                          0
Avg_Temperature_C             0
CO2_Emissions_MMT             0
Deforestation_Rate_Percent    0
dtype: int64

Average Temperature by Year:
Year
2000    19.48
2001    20.14
2002    22.26
2003    25.35
2004    23.41
2005    20.50
2006    22.72
2007    23.92
2008

c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Year 2025: Temp=17.80°C, CO2=2653.57 MMT, Deforestation_Rate_Percent=2.85%
Year 2050: Temp=18.33°C, CO2=2778.68 MMT, Deforestation_Rate_Percent=5.40%
Year 2075: Temp=18.85°C, CO2=2899.70 MMT, Deforestation_Rate_Percent=7.79%
Year 2100: Temp=19.17°C, CO2=3030.13 MMT, Deforestation_Rate_Percent=10.24%
✅ Models trained and saved successfully!


In [9]:
import numpy as np
import joblib

# Step 1: Load the trained models and scalers
try:
    temp_model = joblib.load("temperature_model.pkl")
    co2_model = joblib.load("co2_model.pkl")
    deforestation_model = joblib.load("deforestation_model.pkl")
    label_encoder = joblib.load("label_encoder.pkl")
    scaler = joblib.load("scaler.pkl")
    print("✅ Models and scalers loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    exit()

# Step 2: Display available countries
available_countries = label_encoder.classes_.tolist()
print("\nAvailable Countries:")
print(", ".join(available_countries))

# Step 3: Take user input once
# Input country
country = input("\nEnter a country: ").strip()

# Validate country
if country not in available_countries:
    print(f"Error: Country '{country}' not found. Please choose from the available countries.")
    exit()

# Input year
try:
    year = int(input("Enter a year (1990–2100): ").strip())
    if year < 1990 or year > 2100:
        print("Error: Year must be between 1990 and 2100.")
        exit()
except ValueError:
    print("Error: Year must be a valid integer.")
    exit()

# Step 4: Encode the country and prepare input
country_encoded = label_encoder.transform([country])[0]
input_data = np.array([[country_encoded, year]])
input_data_scaled = scaler.transform(input_data)

# Step 5: Make predictions
predicted_temp = temp_model.predict(input_data_scaled)[0]
predicted_co2 = co2_model.predict(input_data_scaled)[0]
predicted_def = deforestation_model.predict(input_data_scaled)[0]

# Step 6: Display predictions
print(f"\nPredictions for {country} in {year}:")
print(f"Temperature: {predicted_temp:.2f}°C")
print(f"CO2 Emissions: {predicted_co2:.2f} MMT")
print(f"Deforestation Rate: {predicted_def:.2f}%")

✅ Models and scalers loaded successfully!

Available Countries:
Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Mexico, Russia, Saudi Arabia, South Africa, South Korea, Spain, Turkey, United Kingdom, United States

Predictions for Brazil in 2090:
Temperature: 20.97°C
CO2 Emissions: 4608.01 MMT
Deforestation Rate: 8.37%


c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Step 1: Inspect the Dataset
print("Inspecting the Dataset...")
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")

# Rename columns
data.rename(columns={
    "Avg_Temperature_C": "avg_temperature",
    "CO2_Emissions_MMT": "co2_emissions",
    "Country": "country",
    "Deforestation_Rate_Percent": "deforestation_rate",
    "Year": "year"
}, inplace=True)

# Basic info
print("Dataset Info:")
print(data.info())
print("\nYear Range:", data["year"].min(), "to", data["year"].max())
print("\nUnique Countries:", data["country"].nunique())
print("\nMissing Values:")
print(data.isnull().sum())

# Check trends by year
print("\nAverage Temperature by Year:")
print(data.groupby("year")["avg_temperature"].mean().round(2))
print("\nAverage CO2 Emissions by Year:")
print(data.groupby("year")["co2_emissions"].mean().round(2))
print("\nAverage Deforestation Rate by Year:")
print(data.groupby("year")["deforestation_rate"].mean().round(2))

# Step 2: Preprocess Data
# Normalize the country column to handle case sensitivity
data["country"] = data["country"].str.strip().str.title()

# Encode categorical variable (country)
label_encoder = LabelEncoder()
data["country_encoded"] = label_encoder.fit_transform(data["country"])

# Selecting features and target variables
X = data[["country_encoded", "year"]].values  # Convert to numpy array to avoid feature names warning
y_temp = data["avg_temperature"]
y_co2 = data["co2_emissions"]
y_deforestation = data["deforestation_rate"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data (80% train, 20% test)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X_scaled, y_temp, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X_scaled, y_co2, test_size=0.2, random_state=42)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X_scaled, y_deforestation, test_size=0.2, random_state=42)

# Step 3: Train Models
temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)
deforestation_model = RandomForestRegressor(n_estimators=100, random_state=42)

temp_model.fit(X_train, y_temp_train)
co2_model.fit(X_train, y_co2_train)
deforestation_model.fit(X_train, y_deforestation_train)

# Step 4: Evaluate Models
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"--- Model Performance for {target_name} ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print("-" * 40)

print("\nEvaluating Models...")
evaluate_model(temp_model, X_test, y_temp_test, "Temperature")
evaluate_model(co2_model, X_test, y_co2_test, "CO2 Emissions")
evaluate_model(deforestation_model, X_test, y_deforestation_test, "Deforestation Rate")

# Check feature importance
print("Feature Importance (country_encoded, year):")
print("Temperature Model:", temp_model.feature_importances_)
print("CO2 Model:", co2_model.feature_importances_)
print("Deforestation Model:", deforestation_model.feature_importances_)

# Step 5: Test Predictions for United States
sample_country = "United States"
try:
    country_encoded = label_encoder.transform([sample_country])[0]
    test_years = [2025, 2050, 2075, 2100]
    print(f"\nSample Predictions for {sample_country}:")
    for year in test_years:
        input_data = scaler.transform(np.array([[country_encoded, year]]))
        temp_pred = temp_model.predict(input_data)[0]
        co2_pred = co2_model.predict(input_data)[0]
        def_pred = deforestation_model.predict(input_data)[0]
        print(f"Year {year}: Temp={temp_pred:.2f}°C, CO2={co2_pred:.2f} MMT, Deforestation={def_pred:.2f}%")
except ValueError:
    print(f"Country '{sample_country}' not found in dataset")

# Step 6: Save Models and Scaler
joblib.dump(temp_model, "temperature_model.pkl")
joblib.dump(co2_model, "co2_model.pkl")
joblib.dump(deforestation_model, "deforestation_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Models trained and saved successfully!")

Inspecting the Dataset...
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             303 non-null    object 
 1   year                303 non-null    int64  
 2   avg_temperature     303 non-null    float64
 3   co2_emissions       303 non-null    float64
 4   deforestation_rate  303 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 12.0+ KB
None

Year Range: 2000 to 2024

Unique Countries: 20

Missing Values:
country               0
year                  0
avg_temperature       0
co2_emissions         0
deforestation_rate    0
dtype: int64

Average Temperature by Year:
year
2000    19.48
2001    20.14
2002    22.26
2003    25.35
2004    23.41
2005    20.50
2006    22.72
2007    23.92
2008    23.15
2009    24.37
2010    22.20
2011    22.56
2012    26.05
2013    20.46
2014    20.78
20

In [51]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib

# Load Data
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")  # Replace with your file path

# Encode Categorical Variable (Country)
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["Country"])

# Select Features and Target Variables
X = data[["Country_Encoded", "Year"]].values  # Convert to NumPy array
y_temp = data["Avg_Temperature_C"].values
y_co2 = data["CO2_Emissions_MMT"].values
y_deforestation = data["Deforestation_Rate_Percent"].values

# Normalize Data
scaler_x = MinMaxScaler()
scaler_y_temp = MinMaxScaler()
scaler_y_co2 = MinMaxScaler()
scaler_y_deforestation = MinMaxScaler()

X_scaled = scaler_x.fit_transform(X)
y_temp_scaled = scaler_y_temp.fit_transform(y_temp.reshape(-1, 1))
y_co2_scaled = scaler_y_co2.fit_transform(y_co2.reshape(-1, 1))
y_deforestation_scaled = scaler_y_deforestation.fit_transform(y_deforestation.reshape(-1, 1))

# Reshape Data for LSTM (Samples, Time Steps, Features)
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split Data (80% Train, 20% Test)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X_reshaped, y_temp_scaled, test_size=0.2, shuffle=False)
_, _, y_co2_train, y_co2_test = train_test_split(X_reshaped, y_co2_scaled, test_size=0.2, shuffle=False)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X_reshaped, y_deforestation_scaled, test_size=0.2, shuffle=False)

# LSTM Model Function
def build_lstm_model():
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_reshaped.shape[2])),
        Dropout(0.2),
        LSTM(50, activation='relu'),
        Dropout(0.2),
        Dense(1)  # Output layer
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train Models
temp_model = build_lstm_model()
co2_model = build_lstm_model()
deforestation_model = build_lstm_model()

temp_model.fit(X_train, y_temp_train, epochs=50, batch_size=16, validation_data=(X_test, y_temp_test), verbose=1)
co2_model.fit(X_train, y_co2_train, epochs=50, batch_size=16, validation_data=(X_test, y_co2_test), verbose=1)
deforestation_model.fit(X_train, y_deforestation_train, epochs=50, batch_size=16, validation_data=(X_test, y_deforestation_test), verbose=1)

# Save Models
temp_model.save("temperature_lstm_model.h5")
co2_model.save("co2_lstm_model.h5")
deforestation_model.save("deforestation_lstm_model.h5")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler_x, "scaler_x.pkl")
joblib.dump(scaler_y_temp, "scaler_y_temp.pkl")
joblib.dump(scaler_y_co2, "scaler_y_co2.pkl")
joblib.dump(scaler_y_deforestation, "scaler_y_deforestation.pkl")

print("✅ LSTM Models trained and saved successfully!")

# Function to Predict Using LSTM Model
def predict_climate_lstm(country, year):
    # Load models and encoders
    temp_model = tf.keras.models.load_model("temperature_lstm_model.h5")
    co2_model = tf.keras.models.load_model("co2_lstm_model.h5")
    deforestation_model = tf.keras.models.load_model("deforestation_lstm_model.h5")
    
    label_encoder = joblib.load("label_encoder.pkl")
    scaler_x = joblib.load("scaler_x.pkl")
    scaler_y_temp = joblib.load("scaler_y_temp.pkl")
    scaler_y_co2 = joblib.load("scaler_y_co2.pkl")
    scaler_y_deforestation = joblib.load("scaler_y_deforestation.pkl")

    # Convert Country to Encoded Value
    country_encoded = label_encoder.transform([country])[0]

    # Prepare Input
    input_data = np.array([[country_encoded, year]])
    input_scaled = scaler_x.transform(input_data)
    input_reshaped = input_scaled.reshape((1, 1, input_scaled.shape[1]))

    # Make Predictions (Inverse Transform to Original Scale)
    predicted_temp = scaler_y_temp.inverse_transform(temp_model.predict(input_reshaped))[0][0]
    predicted_co2 = scaler_y_co2.inverse_transform(co2_model.predict(input_reshaped))[0][0]
    predicted_deforestation = scaler_y_deforestation.inverse_transform(deforestation_model.predict(input_reshaped))[0][0]

    return {
        "Predicted Temperature (°C)": round(predicted_temp, 2),
        "Predicted CO2 Emissions (MMT)": round(predicted_co2, 2),
        "Predicted Deforestation Rate (%)": round(predicted_deforestation, 2),
    }

# Example Usage
country_name = input("🌍 Enter Country: ")
year_input = int(input("📅 Enter Year: "))

prediction = predict_climate_lstm(country_name, year_input)
print("📊 Predicted Climate Impact:")
print(prediction)


TypeError: unhashable type: 'list'

In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Load Data (Replace 'climate_data.csv' with your actual file)
data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")  # Replace with your file path

# Encode Categorical Variable (Country)
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["Country"])

# Selecting Features and Target Variables
X = data[["Country_Encoded", "Year"]]
y_temp = data["Avg_Temperature_C"]
y_co2 = data["CO2_Emissions_MMT"]
y_deforestation = data["Deforestation_Rate_Percent"]

# Split Data (80% Train, 20% Test)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X, y_co2, test_size=0.2, random_state=42)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X, y_deforestation, test_size=0.2, random_state=42)

# Train Models
temp_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)
deforestation_model = RandomForestRegressor(n_estimators=100, random_state=42)

temp_model.fit(X_train, y_temp_train)
co2_model.fit(X_train, y_co2_train)
deforestation_model.fit(X_train, y_deforestation_train)

# Save Models
joblib.dump(temp_model, "temperature_model.pkl")
joblib.dump(co2_model, "co2_model.pkl")
joblib.dump(deforestation_model, "deforestation_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("✅ Models trained and saved successfully!")

# Function to evaluate model performance
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"--- Model Performance for {target_name} ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print("-" * 40)

# Evaluate models
evaluate_model(temp_model, X_test, y_temp_test, "Temperature")
evaluate_model(co2_model, X_test, y_co2_test, "CO2 Emissions")
evaluate_model(deforestation_model, X_test, y_deforestation_test, "Deforestation Rate")

# Prediction Function
def predict_climate(country, year):
    # Load models and encoder
    temp_model = joblib.load("temperature_model.pkl")
    co2_model = joblib.load("co2_model.pkl")
    deforestation_model = joblib.load("deforestation_model.pkl")
    label_encoder = joblib.load("label_encoder.pkl")

    # Convert Country to Encoded Value
    country_encoded = label_encoder.transform([country])[0]

    # Prepare Input
    input_data = np.array([[country_encoded, year]])

    # Make Predictions
    predicted_temp = temp_model.predict(input_data)[0]
    predicted_co2 = co2_model.predict(input_data)[0]
    predicted_deforestation = deforestation_model.predict(input_data)[0]

    # Display Results
    return {
        "Predicted Temperature (°C)": round(predicted_temp, 2),
        "Predicted CO2 Emissions (MMT)": round(predicted_co2, 2),
        "Predicted Deforestation Rate (%)": round(predicted_deforestation, 2),
    }

# Example Usage
country_name = input("🌍 Enter Country: ")
year_input = int(input("📅 Enter Year: "))

prediction = predict_climate(country_name, year_input)
print("📊 Predicted Climate Impact:")
print(prediction)


✅ Models trained and saved successfully!
--- Model Performance for Temperature ---
Mean Absolute Error (MAE): 6.88
Mean Squared Error (MSE): 69.88
Root Mean Squared Error (RMSE): 8.36
R² Score: -0.29
----------------------------------------
--- Model Performance for CO2 Emissions ---
Mean Absolute Error (MAE): 1131.76
Mean Squared Error (MSE): 1923099.03
Root Mean Squared Error (RMSE): 1386.76
R² Score: -0.09
----------------------------------------
--- Model Performance for Deforestation Rate ---
Mean Absolute Error (MAE): 1.41
Mean Squared Error (MSE): 2.70
Root Mean Squared Error (RMSE): 1.64
R² Score: -0.19
----------------------------------------
📊 Predicted Climate Impact:
{'Predicted Temperature (°C)': np.float64(24.35), 'Predicted CO2 Emissions (MMT)': np.float64(1328.76), 'Predicted Deforestation Rate (%)': np.float64(2.51)}


c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [55]:
import pandas as pd
import numpy as np
import tensorflow as tf
try:
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout
    print("Using standalone keras imports")
except:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    print("Using tensorflow.keras imports")
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Verify imports
print("TensorFlow Version:", tf.__version__)
try:
    import keras
    print("Keras Version:", keras.__version__)
except:
    print("Standalone Keras not installed, using tensorflow.keras")

# Load Data
try:
    data = pd.read_csv(r"C:\Users\Sairam\Desktop\Asha_MITRAz\flask\climate_visualizer\climate_data (1).csv")
    data.rename(columns={
        "Avg_Temperature_C": "avg_temperature",
        "CO2_Emissions_MMT": "co2_emissions",
        "Country": "country",
        "Deforestation_Rate_Percent": "deforestation_rate",
        "Year": "year"
    }, inplace=True)
except FileNotFoundError:
    print("Error: CSV file not found. Please check the file path.")
    exit(1)

# Sort data by country and year
data = data.sort_values(by=["country", "year"])

# Encode Country
label_encoder = LabelEncoder()
data["Country_Encoded"] = label_encoder.fit_transform(data["country"])

# Normalize Features and Targets
scaler_x = MinMaxScaler()
data[["year", "Country_Encoded", "avg_temperature", "co2_emissions", "deforestation_rate"]] = scaler_x.fit_transform(
    data[["year", "Country_Encoded", "avg_temperature", "co2_emissions", "deforestation_rate"]]
)

# Create Sequences for LSTM
def create_sequences(data, seq_length=5):
    X, y_temp, y_co2, y_deforestation = [], [], [], []
    for country in data["country"].unique():
        country_data = data[data["country"] == country].sort_values("year")
        if len(country_data) < seq_length + 1:
            print(f"Warning: Not enough data for {country} (only {len(country_data)} rows)")
            continue
        for i in range(len(country_data) - seq_length):
            seq = country_data[["Country_Encoded", "year", "avg_temperature", "co2_emissions", "deforestation_rate"]].iloc[i:i+seq_length].values
            X.append(seq)
            y_temp.append(country_data["avg_temperature"].iloc[i+seq_length])
            y_co2.append(country_data["co2_emissions"].iloc[i+seq_length])
            y_deforestation.append(country_data["deforestation_rate"].iloc[i+seq_length])
    return np.array(X), np.array(y_temp), np.array(y_co2), np.array(y_deforestation)

seq_length = 5
X, y_temp, y_co2, y_deforestation = create_sequences(data, seq_length)

if X.size == 0:
    print("Error: No sequences created. Check dataset size or sequence length.")
    exit(1)

# Split Data (no shuffling to preserve temporal order)
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, shuffle=False)
_, _, y_co2_train, y_co2_test = train_test_split(X, y_co2, test_size=0.2, shuffle=False)
_, _, y_deforestation_train, y_deforestation_test = train_test_split(X, y_deforestation, test_size=0.2, shuffle=False)

# Build LSTM Model
def build_lstm_model(seq_length, n_features):
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, n_features)),
        Dropout(0.2),
        LSTM(50, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train Models
try:
    temp_model = build_lstm_model(seq_length, X.shape[2])
    co2_model = build_lstm_model(seq_length, X.shape[2])
    deforestation_model = build_lstm_model(seq_length, X.shape[2])

    temp_model.fit(X_train, y_temp_train, epochs=50, batch_size=16, validation_data=(X_test, y_temp_test), verbose=1)
    co2_model.fit(X_train, y_co2_train, epochs=50, batch_size=16, validation_data=(X_test, y_co2_test), verbose=1)
    deforestation_model.fit(X_train, y_deforestation_train, epochs=50, batch_size=16, validation_data=(X_test, y_deforestation_test), verbose=1)
except Exception as e:
    print("Error during model training:", str(e))
    exit(1)

# Save Models
try:
    temp_model.save("temperature_lstm_model.h5")
    co2_model.save("co2_lstm_model.h5")
    deforestation_model.save("deforestation_lstm_model.h5")
    joblib.dump(label_encoder, "label_encoder.pkl")
    joblib.dump(scaler_x, "scaler_x.pkl")
    print("✅ LSTM Models trained and saved successfully!")
except Exception as e:
    print("Error saving models:", str(e))
    exit(1)

# Prediction Function
def predict_climate_lstm(country, year, data, seq_length=5):
    try:
        temp_model = tf.keras.models.load_model("temperature_lstm_model.h5")
        co2_model = tf.keras.models.load_model("co2_lstm_model.h5")
        deforestation_model = tf.keras.models.load_model("deforestation_lstm_model.h5")
        label_encoder = joblib.load("label_encoder.pkl")
        scaler_x = joblib.load("scaler_x.pkl")
    except Exception as e:
        print("Error loading models:", str(e))
        return None
    
    # Ensure country is a string
    if isinstance(country, list):
        country = country[0]
    elif not isinstance(country, str):
        raise ValueError(f"Country must be a string, got {type(country)}")
    
    # Prepare data
    country_data = data[data["country"] == country].sort_values("year").copy()
    if len(country_data) < seq_length:
        raise ValueError(f"Not enough data for {country} to create a sequence of length {seq_length}")
    
    # Normalize year in the data
    country_data[["year", "Country_Encoded", "avg_temperature", "co2_emissions", "deforestation_rate"]] = scaler_x.transform(
        country_data[["year", "Country_Encoded", "avg_temperature", "co2_emissions", "deforestation_rate"]]
    )
    country_data["Country_Encoded"] = label_encoder.transform([country])[0]
    
    # Get sequence for the target year
    target_year_normalized = (year - 2000) / (2023 - 2000)
    current_seq = country_data.tail(seq_length)[["Country_Encoded", "year", "avg_temperature", "co2_emissions", "deforestation_rate"]].values
    
    # Iteratively predict for future years
    while current_seq[-1, 1] < target_year_normalized:
        seq = current_seq.reshape(1, seq_length, 5)
        predicted_temp = temp_model.predict(seq, verbose=0)[0][0]
        predicted_co2 = co2_model.predict(seq, verbose=0)[0][0]
        predicted_deforestation = deforestation_model.predict(seq, verbose=0)[0][0]
        
        new_year = current_seq[-1, 1] + 1 / (2023 - 2000)
        new_row = np.array([current_seq[-1, 0], new_year, predicted_temp, predicted_co2, predicted_deforestation])
        current_seq = np.vstack((current_seq[1:], new_row))
    
    # Final prediction
    seq = current_seq.reshape(1, seq_length, 5)
    predicted_temp = temp_model.predict(seq, verbose=0)[0][0]
    predicted_co2 = co2_model.predict(seq, verbose=0)[0][0]
    predicted_deforestation = deforestation_model.predict(seq, verbose=0)[0][0]
    
    # Inverse transform predictions
    dummy_array = np.zeros((1, 5))
    dummy_array[0, 2] = predicted_temp
    dummy_array[0, 3] = predicted_co2
    dummy_array[0, 4] = predicted_deforestation
    predicted_values = scaler_x.inverse_transform(dummy_array)[:, 2:]
    
    return {
        "Predicted Temperature (°C)": round(predicted_values[0, 0], 2),
        "Predicted CO2 Emissions (MMT)": round(predicted_values[0, 1], 2),
        "Predicted Deforestation Rate (%)": round(predicted_values[0, 2], 2),
    }

# Test Predictions
try:
    country = input("🌍 Enter Country: ")
    year = int(input("📅 Enter Year: "))
    prediction = predict_climate_lstm(country, year, data)
    if prediction:
        print("📊 Predicted Climate Impact:")
        print(prediction)
    
        # Test multiple years
        print(f"\nTesting predictions for {country} across years:")
        for test_year in [2005, 2010, 2015, 2020, 2025]:
            prediction = predict_climate_lstm(country, test_year, data)
            if prediction:
                print(f"Year {test_year}:")
                print(prediction)
except ValueError as e:
    print("Error:", str(e))
except Exception as e:
    print("Unexpected error:", str(e))

TypeError: unhashable type: 'list'